In [0]:
import tensorflow as tf

In [0]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

Cloning into 'fenwicks'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 647 (delta 21), reused 36 (delta 12), pack-reused 600
Receiving objects: 100% (647/647), 815.27 KiB | 6.79 MiB/s, done.
Resolving deltas: 100% (347/347), done.


In [0]:
from IPython.display import Audio
from scipy.io import wavfile
import librosa
import fenwicks as fw
import os
import numpy as np

In [0]:
BUCKET = 'gs://gs_colab'
PROJECT = 'speech_cmd'
BATCH_SIZE = 128 #@param ["128", "256", "512"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
LEARNING_RATE = 0.001 #@param ["0.001", "0.01", "0.1"] {type:"raw"}

In [0]:
fw.colab_tpu.setup_gcs()

In [0]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.SPEECH_CMD_001, './speech001')

In [0]:
example_audio_fn = os.path.join(data_dir_local, 'happy/012c8314_nohash_0.wav')
Audio(example_audio_fn)

In [0]:
SAMPLE_RATE = 16000
NOISE_DIR = '_background_noise_'
data_fn = os.path.join(data_dir, 'speech001_images')

In [0]:
def gen_silence():
  NUM_SEGMENT = 400
  NUM_SYNTH = 500

  path = os.path.join(data_dir_local, NOISE_DIR)
  out_path = os.path.join(data_dir_local, 'silence/')

  fw.io.create_clean_dir(out_path)  
  files = fw.io.enum_files(path, 'wav')
  
  for filename in files:
    _, samples = wavfile.read(filename)
    for i in range(NUM_SEGMENT):
      out_name = f'segment_{i}_{os.path.basename(filename)}'
      data = (samples[i * 200: i * 200 + SAMPLE_RATE] 
              * max(0, 2 * (np.random.random() - 0.25))).astype('int16')
      if data.max() != 0:
        wavfile.write(out_path + out_name, SAMPLE_RATE, data)

  for i in range(NUM_SYNTH):
    d = fw.audio_io.gen_synth_silence(sr = SAMPLE_RATE, n_rand = 4600)
    wavfile.write(os.path.join(out_path, f'new_synthesized_{i}.wav'), SAMPLE_RATE, d)

In [0]:
gen_silence()

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning:

Chunk (non-data) not understood, skipping it.



In [0]:
x_example = fw.audio_io.read_logmelspectrogram(example_audio_fn)
x_example.shape

(40, 101)

In [0]:
paths, y, labels = fw.io.data_dir_tfrecord(data_dir_local, data_fn,
                  extractor=fw.audio_io.read_logmelspectrogram, 
                  file_ext='wav', exclude_dirs=[NOISE_DIR])

In [0]:
n_classes = len(labels)
n_all = len(y)

In [0]:
def build_nn(c=16, kernel_size=(2,5), c_dense=256, drop_rate=0.5):
  model = fw.Sequential()
  model.add(fw.layers.ConvBlk(c, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*2, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*4, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.ConvBlk(c*8, convs=2, kernel_size=kernel_size))
  model.add(fw.layers.GlobalPools())
  model.add(fw.layers.DenseBN(c_dense, drop_rate=drop_rate))
  model.add(fw.layers.DenseBN(c_dense, drop_rate=drop_rate))
  model.add(fw.layers.Classifier(n_classes))
  return model

In [0]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid

total_steps = n_train // BATCH_SIZE * EPOCHS
lr_func = fw.train.cosine_lr(init_lr=LEARNING_RATE, total_steps=total_steps)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func)

In [0]:
parser = lambda x: fw.io.tfexample_numpy_image_parser(x, 40, 101, 1)
est = fw.tpuest.get_tpu_estimator(n_train, n_valid, model_func, work_dir, trn_bs=BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/speech_cmd/2019-04-15-14:49:13', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.27.213.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f11336f25c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.27.213.82:8470', '_evaluation_master': 'grpc://10.27.213.82:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=418, num_shards=No

In [0]:
def train_eval_fold(val_fold):
  train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=True)
  valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser,
    params['batch_size'], n_folds=5, val_fold_idx = val_fold, training=False)

  est.train(train_input_func, steps=total_steps)
  result = est.evaluate(valid_input_func, steps=1)
  fw.io.create_clean_dir(work_dir)
  return result

In [0]:
result = []
result.append(train_eval_fold(0))

INFO:tensorflow:Querying Tensorflow master (grpc://10.27.213.82:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6226888201725444236)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5685808977411904767)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1062720204702421928)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 378741869925337482)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14151405457217059618)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/j

In [0]:
result.append(train_eval_fold(1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-15-14:49:13/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 9 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss =

In [0]:
result.append(train_eval_fold(2))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-15-14:49:13/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
result.append(train_eval_fold(3))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-15-14:49:13/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 9 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss =

In [0]:
result.append(train_eval_fold(4))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://gs_colab/work/speech_cmd/2019-04-15-14:49:13/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (418) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (418) batch(es) of data from outfeed.
INFO:tensorflow:loss 

In [0]:
result

[{'accuracy': 0.9708955, 'global_step': 10032, 'loss': 0.09185194},
 {'accuracy': 0.9722388, 'global_step': 10032, 'loss': 0.13105705},
 {'accuracy': 0.97126865, 'global_step': 10032, 'loss': 0.10088032},
 {'accuracy': 0.96970147, 'global_step': 10032, 'loss': 0.14378482},
 {'accuracy': 0.9708209, 'global_step': 10032, 'loss': 0.12851356}]

In [0]:
acc = [res['accuracy'] for res in result]
loss = [res['loss'] for res in result]

print('accuracy:', np.mean(acc), '+/-', np.std(acc))
print('loss:', np.mean(loss), '+/-', np.std(loss))

accuracy: 0.97098505 +/- 0.00081667694
loss: 0.11921754 +/- 0.019571772
